In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from collections import OrderedDict
def get_sports():
    df_sport_latest = pd.read_csv('articles/sports_articles.csv', encoding = "ISO-8859-1")
    df_sport_latest_tv2 = pd.read_csv('articles/sports_articles_tv2.csv', encoding = "ISO-8859-1")
    df_sport_2019 = pd.read_csv('articles/sports_articles_2019.csv', encoding = "ISO-8859-1")
    df_sport_2020 = pd.read_csv('articles/sports_articles_2020.csv', encoding = "ISO-8859-1")
    df_sport_2022 = pd.read_csv('articles/sports_articles_2022.csv', encoding = "ISO-8859-1")
    df = pd.concat([df_sport_latest, df_sport_latest_tv2, df_sport_2019, df_sport_2020, df_sport_2022])
    df = df.sample(frac=1).reset_index(drop=True)
    return df


In [ ]:
def vocab_2_pdset(columns, df):
    df_vocab_select_columns = df.iloc[:, columns]
    vocab_all_values = df_vocab_select_columns.values.ravel()
    return set(vocab_all_values)

def vocab_2_dict(sets):
    assert(len(sets) == 4)
    word_set = sets[0].union(sets[1],sets[2], sets[3])
    df = pd.DataFrame(list(word_set), columns=["Words"])
    df.sort_values(by="Words", inplace=True)
    df.reset_index(drop=True, inplace=True)
    return OrderedDict.fromkeys(word_set)

def get_vocab_dict():
    df_ods_vocab = pd.read_table('ods_fullforms_2020-08-26.csv', header=None)
    df_ddo_vocab = pd.read_table('ddo_fullforms_2020-08-26.csv', header=None)
    df_vocab = pd.read_table('cor1.02.tsv', header=None)
    df_sport_lingo = pd.read_table('sport_lingo.csv', header=None)

    vocab_set = vocab_2_pdset([1,3], df_vocab)
    ods_vocab_set = vocab_2_pdset([0,1], df_ods_vocab)
    ddo_vocab_set = vocab_2_pdset([0,1], df_ddo_vocab)
    sport_lingo_set = vocab_2_pdset([0], df_sport_lingo)

    return vocab_2_dict([vocab_set, ods_vocab_set, ddo_vocab_set, sport_lingo_set])

ordered_dict = get_vocab_dict()


In [ ]:
df_sport = get_sports()

In [ ]:
import time
# isin_dict = False
def test_lookup_performance():
    word_to_check = "linebreak"
    start_time = time.time()

    for x in range(1000000):
        isin_dict = word_to_check in ordered_dict

    end_time = time.time()  
    assert(end_time - start_time < 1)
    print(isin_dict)

test_lookup_performance()

# isin_dict


In [ ]:
import nltk as nltk
from string import punctuation
from nltk.corpus import stopwords
nltk.download('stopwords')

da_stopwords = stopwords.words("danish")





In [ ]:
import re


df_sport_text = df_sport.iloc[:, [0,1,2]]

train_text = df_sport_text.apply(' '.join, axis=1).to_numpy()

words_arr = []

def replace_digits(word):
    return re.sub(r'\d+', 'X', word)

def split_word(word):
    characters_to_remove = ["-"]
    new_word = word

    for char in characters_to_remove:
        new_word = new_word.replace(char, ' ')
    return new_word.split()

def remove_specials(word):
    characters_to_remove = [':', "'", '?', ",", ".", "(", ")"]
    new_word = word

    for char in characters_to_remove:
        new_word = new_word.replace(char, '')
    return new_word

def contains_non_alphanumeric(word):
    return bool(re.search(r'[^a-zæøåA-ZÆØÅ0-9]', word))

def formatWord(word):
    if any(char.isdigit() for char in word):
        return replace_digits(word)
    
    
for sentences in range(len(train_text)):
    # print(sport_vocab[sentences])
    sentence = train_text[sentences].strip()
    words = sentence.split()
    for word in range(len(words)):
        w = words[word]
        if contains_non_alphanumeric(w):
            w = remove_specials(w)
            w = split_word(w)
            words_arr.extend([x.lower() for x in w])
        else:
            words_arr.append(w.lower())


words_sport_unique = set(words_arr)
words_sport_unique_list = list(words_sport_unique)
words_sport_lingo = []
words_train_vocab = []

# TODO : brug tensorflow Tokenezier til at omdanne ord til tokens
# TODO : søg i alle leksikoner, søg med og uden bindestreg
# TODO : håndter tal ikke i ordbøger eks ( x-x eller x-årig)
# TODO : lemmatizer : udelad bøjninger af samme navneord. eks : verdensmester/verdensmesteren
# TODO : evt. grupper ord der ofte hænger sammen med nltk BigramFinder. eks vandt over

for w in range(len(words_sport_unique_list)):
    word = words_sport_unique_list[w]
    if any(char.isdigit() for char in word):
        words_train_vocab.append(word)
    else: 
        isin_dict = word in ordered_dict
        if (isin_dict == False):
            words_sport_lingo.append(word)
        else:
            words_train_vocab.append(word)

print("total unique words:", len(words_sport_unique) )
print("total sports lingo words:", len(words_sport_lingo) )
print("total vocab:", len(words_train_vocab))
print("total articles:", len(df_sport) )


In [ ]:
# def remove_stopwords(word):
#     if word in da_stopwords:
#         return True
#     else:
#         return False

# words_train_vocab = [x for x in words_train_vocab if not remove_stopwords(x)]

# words_train_vocab


In [ ]:
# from collections import Counter
# cnt = Counter()
# for word in words_train_vocab:
#    cnt[word] += 1

# s = sorted(cnt.items(), key=lambda item: item[1])
# s.reverse()

# s


In [ ]:
words_sport_lingo
file = open('words_sport_lingo.txt','w')
for item in words_sport_lingo:
	file.write(item+"\n")
file.close()


file = open('words_train_vocab.txt','w')
for item in sorted(words_train_vocab):
	file.write(item+"\n")
file.close()

In [ ]:
def format_2_bool(x):
    if type(x) == bool:
        return x
    assert(type(x) == str)
    x_copy = x
    x_copy = x_copy.strip()
    x_copy = x_copy.lower()
    assert(x_copy == "true" or x_copy == "false")
    if x_copy == "true":
        return True
    else:
        return False

In [ ]:

df_sport_labels = df_sport['isResult'].apply(lambda x: format_2_bool(x))

results_true = df_sport_labels.loc[df_sport_labels== True]
results_false = df_sport_labels.loc[df_sport_labels == False]

assert(len(results_true) + len(results_false) == len(df_sport_labels))

print(len(results_true))
print(len(results_false))
labels = df_sport_labels.to_numpy().astype(int)




In [ ]:
# df_sport_text = df_sport.iloc[:, [0,1,2]]


# df_sport_text
# df_sport_text_combined = df_sport_text.apply(' '.join, axis=1)
# train_text = df_sport_text.apply(' linebreak '.join, axis=1).to_numpy()



print(len(labels))
print(len(train_text))
print(labels.shape)
print(train_text.shape)
longest_text = len(max(train_text, key=len))
print("longest text: ", longest_text)
train_text[0]

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re

def split_dash(input_data):
  return tf.strings.regex_replace(input_data, '-', ' ')

def replace_digits(word):
    return tf.strings.regex_replace(word, pattern=r'\d+', rewrite=r'X')

def remove_specials(word):
    return tf.strings.regex_replace(word, pattern=r'[:,\'\.]', rewrite=r'')

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    replaced_digits = replace_digits(lowercase)
    removed_specials = remove_specials(replaced_digits)
    split_dashed = split_dash(removed_specials)
    return split_dashed


# Model constants.
max_features = 2700
sequence_length = longest_text + 2

# Now that we have our custom standardization, we can instantiate our text
# vectorization layer. We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

text_ds = vectorize_layer.adapt(words_train_vocab)

vect_vocab = vectorize_layer.get_vocabulary()

text_vec = vectorize_layer([train_text[0]])

def vect_layer_2_text(vect_l):
    return np.array([vect_vocab[x] for x in np.trim_zeros(np.squeeze(vect_l.numpy()))])


# text = vect_layer_2_text(text_vec)

print(len(vect_vocab))
print(vect_layer_2_text(text_vec))

vect_vocab



In [ ]:


train_ds = vectorize_layer(train_text)

train_data = train_ds[0:420]
val_data = train_ds[420:]

train_labels = labels[0:420]
val_labels = labels[420:]

train_data
# val
# labels.shape

In [ ]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions



In [ ]:
from tensorflow.keras import layers
import random as python_random

embed_dim = 96  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer


# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embed_dim)(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
# x = layers.Dropout(0.1)(x)

# Conv1D + global max pooling
# x = layers.Conv1D(128, 10, padding="valid", activation="relu", strides=3)(x)
# x = layers.Conv1D(128, 10, padding="valid", activation="relu", strides=3)(x)


x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
# x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

transformer_model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
transformer_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
epochs = 8
# Fit the model using the train and test datasets.
transformer_model.fit(train_data, train_labels, epochs=epochs, batch_size=6, validation_data=(val_data, val_labels))

In [ ]:
from tensorflow.keras import layers
import random as python_random

embedding_dim = 96

# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 10, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 10, padding="valid", activation="relu", strides=3)(x)


x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
# x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:

epochs = 8
# Fit the model using the train and test datasets.
model.fit(train_data, train_labels, epochs=epochs, batch_size=6, validation_data=(val_data, val_labels))

In [ ]:
score = model.evaluate(val_data, val_labels, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
np.set_printoptions(precision = 2, suppress = True)
predictions = model.predict(val_data)
print("Number of predictions: ", len(predictions))


In [ ]:
num_correct = 0
for x in range(len(val_data)):
    n_pred = np.rint(np.squeeze(predictions[x])).astype(int)
    n_label = np.rint(val_labels[x]).astype(int)
    num_correct += 1 if n_pred == n_label else 0

print("Predictions correct", (num_correct / len(predictions) ) * 100)

In [ ]:
for x in range(len(val_data)):
    print("VALIDATION SAMPLE: \n" ,vect_layer_2_text(val_data[x]))
    print("LABEL --:" , val_labels[x])
    print("PREDICTION --:" , predictions[x])
    print("\n")


In [ ]:
import os
from tensorboard.plugins import projector
# Set up a logs directory, so Tensorboard knows where to look for files.

ll = model.layers[1]
ll_weights = ll.get_weights()[0]

print(ll_weights.shape)
ll_weights


In [ ]:
##import I/O module in python
import io

##open the text stream for vectors
vectors = io.open('vectors.tsv', 'w', encoding='utf-8')

##open the text stream for metadata
meta = io.open('meta.tsv', 'w', encoding='utf-8')


##write each word and its corresponding embedding
for index in range(1, len(vect_vocab)):
  word = vect_vocab[index]  # flipping the key-value in word_index
  embeddings = ll_weights[index]
  meta.write(word + "\n")
  vectors.write('\t'.join([str(x) for x in embeddings]) + "\n")

##close the stream
vectors.close()
meta.close()

In [ ]:
from nltk import collocations
bigram_measures = collocations.BigramAssocMeasures()
finder = collocations.BigramCollocationFinder.from_words(["New", "York", "is", "big", "New", "York", "is", "pretty"])
finder.ngram_fd.items()



In [ ]:
import lemmy
# Create an instance of the standalone lemmatizer.
lemmatizer = lemmy.load("da")

# Find lemma for the word 'akvariernes'. First argument is an empty POS tag.
lemmatizer.lemmatize("NOUN", "storsejr")

